In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import pandas as pd
import seaborn as sns
import sklearn

# Описание работы
Измерение удовлетворенности клиентов является ключевым фактором для ведения современного бизнеса, поскольку оно может внести значительный вклад в повышение качества обслуживания. Чтобы соответствовать ожиданиям клиентов и достигать более высокого уровня качества, авиакомпаниям необходимо разработать конкретный механизм измерения удовлетворенности использования предоставленных авикомпанией услуг пассажиров. Таким образом, данная работа решает задачу классификацию удовлетворенности клиентов авиакомпаний.

## 1. Загрузка и чтение данных

In [2]:
url = "https://github.com/a-sidorova/machine-learning-course/raw/master/airline_passenger_satisfaction.csv"
data_raw = pd.read_csv(url)
data_raw.shape

(129880, 24)

Таблица содержит информацию о порядка 129 тысяч пассажиров авиаперелетов. 

In [3]:
data_raw.head()

,Unnamed: 0,Gender,customer_type,age,type_of_travel,customer_class,flight_distance,inflight_wifi_service,departure_arrival_time_convenient,ease_of_online_booking,...,inflight_entertainment,onboard_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,satisfaction
0,0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


Каждый пассажир охарактеризован 24 признаками. Среди них есть категориальные (пол, тип клиента, тип полета, класс клиента, удовлетворенность), числовые (возраст, дистанция перелета, задержка вылета и прилета) и оценки (на сколько удовлетворен клиент какой-либо услугой).

Рассмотрим отдельно каждый признак:


In [4]:
data_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129880 entries, 0 to 129879
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         129880 non-null  int64  
 1   Gender                             129880 non-null  object 
 2   customer_type                      129880 non-null  object 
 3   age                                129880 non-null  int64  
 4   type_of_travel                     129880 non-null  object 
 5   customer_class                     129880 non-null  object 
 6   flight_distance                    129880 non-null  int64  
 7   inflight_wifi_service              129880 non-null  int64  
 8   departure_arrival_time_convenient  129880 non-null  int64  
 9   ease_of_online_booking             129880 non-null  int64  
 10  gate_location                      129880 non-null  int64  
 11  food_and_drink                     1298

1. Unnamed: 0 - ID пассажира - числовой (в последствии переименуем)
2. Gender - пол пассажира - категориальный (male, female)
3. cutomer_type - тип клиента - категориальный (лояльный или нет)
4. type_of_travel - тип поездки - категориальный (личная, бизнес)
5. customer_class - класс клиента - категориальный (эконом, эконом+, бизнес)
6. flight_distance - дистанция перелета в миллях - числовой
7. inflight_wifi_service - качество Wi-Fi во время полета - категориальный (1-5)
8. departure_arrival_time_convenient - удобное время вылета и прибытия - категориальный (1-5)
9. ease_of_online_booking - простота онлайн-бронирования - категориальный (1-5)
10. gate_location - удобство расположения выхода - категориальный (1-5)
11. food_and_drink - качество еды - категориальный (1-5)
12. online_boarding - удобство онлайн регистрации - категориальный (1-5)
13. seat_comfort - комфорт сидения - категориальный (1-5)
14. inflight_entertainment - качество разлвечений во время полета - категориальный (1-5)
15. onboard_service - качество услуг на борту - категориальный (1-5)
16. leg_room_service
17. baggage_handling - качество перевоза багажа - категориальный (1-5)
18. checkin_service - качество работы службы регистрации - категориальный (1-5)
19. inflight_service - качество работы бортовой службы - категориальный (1-5)
20. cleanliness - качество чистоты на борту - категориальный (1-5)
21. departure_delay_in_minutes - задержка вылета рейса в минутах - числовой
22. arrival_delay_in_minutes - задержка прилета рейса в минутах - числовой
23. satisfaction - удовлетворенность клиента - категориальный (удовлетворен, нейтрален/неудовлетворен)

Переименуем первую колонку, отвечающую за ID клиента, в более понятное наименование (Unnamed: 0 - ни оч чем нам не говорит):


In [17]:
data_raw.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)
data_raw.columns

Index(['ID', 'Gender', 'customer_type', 'age', 'type_of_travel',
       'customer_class', 'flight_distance', 'inflight_wifi_service',
       'departure_arrival_time_convenient', 'ease_of_online_booking',
       'gate_location', 'food_and_drink', 'online_boarding', 'seat_comfort',
       'inflight_entertainment', 'onboard_service', 'leg_room_service',
       'baggage_handling', 'checkin_service', 'inflight_service',
       'cleanliness', 'departure_delay_in_minutes', 'arrival_delay_in_minutes',
       'satisfaction'],
      dtype='object')

Заменим тип Object категориальных признаков на специальный тип, который предназначен для хранения как раз таких категориальных значений:

In [18]:
data_raw['Gender'] = data_raw['Gender'].astype('category')
data_raw['customer_type'] = data_raw['customer_type'].astype('category')
data_raw['type_of_travel'] = data_raw['type_of_travel'].astype('category')
data_raw['customer_class'] = data_raw['customer_class'].astype('category')
data_raw['satisfaction'] = data_raw['satisfaction'].astype('category')


In [19]:
data_raw['Gender'].dtype

CategoricalDtype(categories=['Female', 'Male'], ordered=False)

In [20]:
data_raw['customer_type'].dtype

CategoricalDtype(categories=['Loyal Customer', 'disloyal Customer'], ordered=False)

In [21]:
data_raw['type_of_travel'].dtype

CategoricalDtype(categories=['Business travel', 'Personal Travel'], ordered=False)

In [22]:
data_raw['customer_class'].dtype

CategoricalDtype(categories=['Business', 'Eco', 'Eco Plus'], ordered=False)

In [23]:
data_raw['satisfaction'].dtype

CategoricalDtype(categories=['neutral or dissatisfied', 'satisfied'], ordered=False)

## 2. Визуализация и основные характеристики

Получим информацию о количественных признаках:

In [24]:
data_raw.describe()

,ID,age,flight_distance,inflight_wifi_service,departure_arrival_time_convenient,ease_of_online_booking,gate_location,food_and_drink,online_boarding,seat_comfort,inflight_entertainment,onboard_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes
count,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129487.000000
mean,64939.500000,39.427957,1190.316392,2.728696,3.057599,2.756876,2.976925,3.204774,3.252633,3.441361,3.358077,3.383023,3.350878,3.632114,3.306267,3.642193,3.286326,14.713713,15.091129
std,37493.270818,15.119360,997.452477,1.329340,1.526741,1.401740,1.278520,1.329933,1.350719,1.319289,1.334049,1.287099,1.316252,1.180025,1.266185,1.176669,1.313682,38.071126,38.465650
min,0.000000,7.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,32469.750000,27.000000,414.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000
50%,64939.500000,40.000000,844.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,0.000000,0.000000
75%,97409.250000,51.000000,1744.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000,4.000000,12.000000,13.000000
max,129879.000000,85.000000,4983.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1592.000000,1584.000000


Таким образом, мы получили количество значений, средние значения, стандартные отклонения, минимальные и максимальные значения, значения квартилей и медианы для количественных признаков.

Проделаем то же самое и для категориальный признаков:

In [26]:
data_raw.describe(include=['category'])

,Gender,customer_type,type_of_travel,customer_class,satisfaction
count,129880,129880,129880,129880,129880
unique,2,2,2,3,2
top,Female,Loyal Customer,Business travel,Business,neutral or dissatisfied
freq,65899,106100,89693,62160,73452


Для категориальных признаков мы получили такие характеристики, как количество значений, количество уникальных значений, наиболее встречающееся значение и его частота.